# Cleaning Data Exercises

In this exercise, we'll be returning to the American Community Survey data we used previously to measuring racial income inequality in the United States. In today's exercise, we'll be using it to measure the returns to education and how those returns vary by race and gender.




## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_cleaning.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}
```


### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

## Exercises

### Exercise 1

For these cleaning exercises, we'll return to the ACS data we've used before one last time. We'll be working with `US_ACS_2017_10pct_sample.dta`. Import the data (please use url for the autograder).

In [16]:
import numpy as np
import pandas as pd

pd.set_option("mode.copy_on_write", True)

In [17]:
acs = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)
acs.head()

,year,datanum,serial,cbserial,numprec,subsamp,hhwt,hhtype,cluster,adjust,...,migcounty1,migmet131,vetdisab,diffrem,diffphys,diffmob,diffcare,diffsens,diffeye,diffhear
0,2017,1,177686,2.017001e+12,9,64,55,"female householder, no husband present",2.017002e+12,1.011189,...,0,not in identifiable area,n/a,n/a,n/a,n/a,n/a,no vision or hearing difficulty,no,no
1,2017,1,1200045,2.017001e+12,6,79,25,"male householder, no wife present",2.017012e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
2,2017,1,70831,2.017000e+12,1 person record,36,57,"male householder, living alone",2.017001e+12,1.011189,...,0,not in identifiable area,n/a,has cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
3,2017,1,557128,2.017001e+12,2,10,98,married-couple family household,2.017006e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
4,2017,1,614890,2.017001e+12,4,96,54,married-couple family household,2.017006e+12,1.011189,...,0,not in identifiable area,n/a,n/a,n/a,n/a,n/a,no vision or hearing difficulty,no,no


### Exercise 2

For our exercises today, we'll focus on `age`, `sex`, `educ` (education), and `inctot` (total income). Subset your data to those variables, and quickly look at a sample of 10 rows.

In [18]:
acs2 = acs[["age", "sex", "educ", "inctot"]]

### Exercise 3

As before, all the values of `9999999` have the potential to cause us real problems, so replace all the values of `inctot` that are `9999999` with `np.nan`. 

In [19]:
acs2 = acs2.replace(9999999, np.nan)

### Exercise 4

Attempt to calculate the average age of people in our data. What do you get? Why are you getting that error?

You *should* get an error in trying to answer this question, but **PLEASE LEAVE THE CODE THAT GENERATES THIS ERROR COMMENTED OUT SO YOUR NOTEBOOK WILL RUN IN THE AUTOGRADER**. 

Then talk about the error in a markdown cell.

In [20]:
# acs2["age"].mean()

The error shows us that age is a categorical variable and it does not support numeric operations. 

### Exercise 5

We want to be able to calculate things using age, so we need it to be a numeric type. Check the current type of `age`, and look at all the values of `age` to figure out why it's categorical and not numeric. You should find two problematic categories. Store the values of these categories in `"ex5_age_young"` and `"ex5_age_old"` (once you find them, it should be clear which is which).

In [21]:
acs2["age"].dtypes

CategoricalDtype(categories=['less than 1 year old', '1', '2', '3', '4', '5', '6', '7',
                  '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
                  '18', '19', '20', '21', '22', '23', '24', '25', '26', '27',
                  '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
                  '38', '39', '40', '41', '42', '43', '44', '45', '46', '47',
                  '48', '49', '50', '51', '52', '53', '54', '55', '56', '57',
                  '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
                  '68', '69', '70', '71', '72', '73', '74', '75', '76', '77',
                  '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
                  '88', '89', '90 (90+ in 1980 and 1990)', '91', '92', '93',
                  '94', '95', '96'],
, ordered=True, categories_dtype=object)

In [22]:
acs2.loc[~acs2["age"].str.isnumeric()]

,age,sex,educ,inctot
37,less than 1 year old,female,n/a or no schooling,NaN
90,less than 1 year old,female,n/a or no schooling,NaN
140,less than 1 year old,female,n/a or no schooling,NaN
179,less than 1 year old,female,n/a or no schooling,NaN
359,90 (90+ in 1980 and 1990),female,1 year of college,6000.0
...,...,...,...,...
318748,less than 1 year old,male,n/a or no schooling,NaN
318849,less than 1 year old,male,n/a or no schooling,NaN
318913,less than 1 year old,female,n/a or no schooling,NaN
318971,less than 1 year old,male,n/a or no schooling,NaN


In [23]:
results = {}
results["ex5_age_young"] = "less than 1 year old"
results["ex5_age_old"] = "90+ in 1980 and 1990"

### Exercise 6

In order to convert `age` into a numeric variable, we need to replace those problematic entries with values that `pandas` can later convert into numbers. Pick appropriate substitutions for the existing values and replace the current values. 

**Hint 1:** Categorical variables act like strings, so you might want to use string methods! 

**Hint 2:** Remember that characters like parentheses, pluses, asterisks, etc. are special in Python strings, and you have to escape them if you want them to be interpreted literally!

**Hint 3:** Because the US Census has been conducted regularly for hundreds of years but exactly how the census has been conducted have occasionally changed, variables are sometimes coded in a way that might be interpreted in different ways for different census years. For example, hypothetically, one might write `90 (90+ in 1980 and 1990)` if the Censuses conducted in 1980 and 1990 used to top-code age at 90 (any values *over* 90 were just coded as 90), but more recent Censuses no longer top-coded age and recorded ages over 90 as the respondents actual age.

In [ ]:
acs2["age"] = acs2["age"].astype(str)
acs2["age"] = acs2["age"].str.replace("less than 1 year old", "0")
acs2["age"] = acs2["age"].str.replace("90 (90+ in 1980 and 1990)", "90")

### Exercise 7

Now convert age from a categorical to numeric. Calculate the average age amoung this group, and store it in `"ex7_avg_age"`.

In [ ]:
acs2["age"] = pd.to_numeric(acs2["age"], errors="coerce")
results["ex7_avg_age"] = acs2["age"].mean()
print(f"The average for all ages in the US Census is {results['ex7_avg_age']:.2f}.")

The average for all ages in the US Census is 41.30.


### Exercise 8

Let's now filter out anyone in our data whose age is less than 18. Note that before made `age` a numeric variable, we couldn't do this! Again, calculate the average age and this time store it in `"ex8_avg_age"`. 

Use this sample of people 18 and over for all subsequent exercises.

In [ ]:
acs3 = acs2[acs2["age"] >= 18].copy()
results["ex8_avg_age"] = acs3["age"].mean()
print(
    f"The average mean for people over the age of 18 is {results['ex8_avg_age']:.2f}."
)

The average mean for people over the age of 18 is 49.76.


### Exercise 9

Create an indicator variable for whether each person has *at least* a college Bachelor's degree called `college_degree`. Use this variable to calculate the number of people in the dataset with a college degree. You may assume that to get a college degree you need to complete at least 4 years of college. Save the result as `"ex9_num_college"`.

In [ ]:
acs3["college_degree"] = acs3["educ"].isin(
    ["4 years of college", "5+ years of college"]
)
results["ex9_num_college"] = acs3["college_degree"].sum()
print(
    f"The number of people in the dataset with a college degree is {results['ex9_num_college']}."
)

The number of people in the dataset with a college degree is 77013.


### Exercise 10

Let's examine how the educational gender gap. Use `pd.crosstab` to create a cross-tabulation of `sex` and `college_degree`. `pd.crosstab` will give you the number of people who have each combination of `sex` and `college_degree` (so in this case, it will give us a 2x2 table with Male and Female as rows, and `college_degree` True and False as columns, or vice versa. 

In [74]:
ct_count = pd.crosstab(acs3["sex"], acs3["college_degree"])

### Exercise 11

Counts are kind of hard to interpret. `pd.crosstab` can also normalize values to give percentages. Look at the `pd.crosstab` help file to figure out how to normalize the values in the table. Normalize them so that you get the share of men with and without college degree, and the share of women with and without college degrees.

Store the share (between 0 and 1) of men with college degrees in `"ex11_share_male_w_degrees"`, and the share of women with degrees in `"ex11_share_female_w_degrees"`.

In [ ]:
ct_all = pd.crosstab(acs3["sex"], acs3["college_degree"], normalize="index")
results["ex11_share_male_w_degrees"] = ct_all.loc["male", True]
results["ex11_share_female_w_degrees"] = ct_all.loc["female", True]
print(ct_all)
print(
    f"Share of men with college degrees is {results['ex11_share_male_w_degrees']:.2f}."
)
print(
    f"Share of women with college degrees is {results['ex11_share_female_w_degrees']:.2f}."
)

college_degree     False     True 
sex                               
male            0.703439  0.296561
female          0.688381  0.311619
Share of men with college degrees is 0.30.
Share of women with college degrees is 0.31.


### Exercise 12

Now, let's recreate that table for people who are 40 and over and people under 40. Over time, what does this suggest about the absolute difference in the share of men and women earning college degrees? Has it gotten larger, stayed the same, or gotten smaller? Store your answer (either `"the absolute difference has increased"` or `"the absolute difference has decreased"`) in `"ex12_comparing"`.

In [72]:
acs3["age_group"] = np.where(acs3["age"] >= 40, "40_and_over", "under_40")

ct_by_age = pd.crosstab(
    [acs3["age_group"], acs3["sex"]], acs3["college_degree"], normalize="index"
)

In [80]:
ct_by_age

college_degree         False     True 
age_group   sex                       
40_and_over male    0.682123  0.317877
            female  0.699144  0.300856
under_40    male    0.743143  0.256857
            female  0.665710  0.334290

In [ ]:
abs_diff_under40 = abs(
    ct_by_age.loc[("under_40", "male"), True]
    - ct_by_age.loc[("under_40", "female"), True]
)
abs_diff_above40 = abs(
    ct_by_age.loc[("40_and_over", "male"), True]
    - ct_by_age.loc[("40_and_over", "female"), True]
)

print(f"Difference under 40 is {abs_diff_under40}")
print(f"Difference above 40 is {abs_diff_above40}")

results["ex12_comparing"] = "the absolute difference has increased"
print(f"Overtime {results["ex12_comparing"]}.")

Difference under 40 is 0.07743230514952698
Difference above 40 is 0.017020152658337562
Overtime the absolute difference has increased.


### Exercise 13

In words, what is causing the change noted in Exercise 12 (i.e., looking at the tables above, tell me a story about Men and Women's College attainment).


In the older generation (40 and over), men and women had roughly equal rates of college attainment, with about 32% of men and 30% of women holding a degree. Among younger adults (under 40), the pattern shifts: only 25.7% of men have a college degree, compared with 33.4% of women. This indicates that women’s college attainment has increased faster than men’s over time, resulting in a larger absolute gender gap in the younger generation.

## Want More Practice?

Calculate the educational racial gap in the United States for White Americans, Black Americans, Hispanic Americans, and other groups. 

Note that to do these calculations, you'll have to deal with the fact that unlike most Americans, the American Census Bureau treats "Hispanic" not as a racial category, but a linguistic one. As a result, the racial category "White" in `race` actually includes most Hispanic Americans. For this analysis, we wish to work with the mutually exclusive categories of "White, non-Hispanic", "White, Hispanic", "Black (Hispanic or non-Hispanic)", and a category for everyone else. 